# Example: Import district heating network from GeoJSON files
============================================================

This example demonstrates how to import a district heating network
from three GeoJSON files representing network topology, buildings,
and supply points.


In [1]:

import os
from uesgraphs import UESGraph
from uesgraphs.examples import e1_readme_example as e1

#Set workspace
workspace = e1.workspace_example("e15")


# Get path to example data
uesgraphs_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(uesgraphs_dir, 'data','examples', 'e15_geojson')

# Create graph and import GeoJSON
graph = UESGraph()
graph.from_geojson(
    network_path=os.path.join(data_dir, 'network.geojson'),
    buildings_path=os.path.join(data_dir, 'buildings.geojson'),
    supply_path=os.path.join(data_dir, 'supply.geojson'),
    name='simple_district',
    save_path=workspace,
    generate_visualizations=True
)


print(f"============ Uesgraphs Import from GeoJSON successful! ============")
print(f"Network nodes: {graph.number_of_nodes('heating')}")
print(f"Buildings: {graph.number_of_nodes('building')}")
print(f"Total length: {graph.network_length:.2f} m")
print(f"Visualizations of network generation can be found in: {os.path.join(workspace, 'visualizations')}")

graph.to_json(os.path.join(workspace, 'simple_district_graph.json'))

Ignoring fixed y limits to fulfill fixed data aspect with adjustable data limits.


Logfile findable here: /tmp/Visuals_20251031_180839.log
Saved visualization to /home/leon/git/uesgraphs/workspace/e15/visuals_of_uesgraph_creation/1_basic_uesgraph.pdf
Logfile findable here: /tmp/Visuals_20251031_180839.log


Ignoring fixed y limits to fulfill fixed data aspect with adjustable data limits.
Ignoring fixed y limits to fulfill fixed data aspect with adjustable data limits.


Saved visualization to /home/leon/git/uesgraphs/workspace/e15/visuals_of_uesgraph_creation/2_with_supply.pdf
Logfile findable here: /tmp/Visuals_20251031_180839.log
Saved visualization to /home/leon/git/uesgraphs/workspace/e15/visuals_of_uesgraph_creation/3_with_bldg.pdf
Logfile findable here: /tmp/Visuals_20251031_180839.log


Ignoring fixed y limits to fulfill fixed data aspect with adjustable data limits.


Saved visualization to /home/leon/git/uesgraphs/workspace/e15/visuals_of_uesgraph_creation/4_coords_transformed.pdf
Total network length (m):  426.27
============ Uesgraphs Import from GeoJSON successful! ============
Network nodes: 8
Buildings: 5
Total length: 426.27 m
Visualizations of network generation can be found in: /home/leon/git/uesgraphs/workspace/e15/visualizations


'/home/leon/git/uesgraphs/workspace/e15/simple_district_graph.json/nodes.json'

In [2]:
"""
Parameter assignment functions for uesgraph components (pipes, supply, demand).

Balanced approach:
- Clean helper functions for maintainability
- Optional logger for debugging and status messages
- Simple return value (only uesgraph)
- Exceptions for errors, warnings for non-critical issues
- Support @ references for linking Excel parameters to node/edge attributes
"""

import os
import logging
import warnings
from uesgraphs.systemmodels.templates import UESTemplates


# ============================================================================
# HELPER FUNCTIONS (shared by all assignment functions)
# ============================================================================

def resolve_parameter_value(value, component_data, param_name, component_id):
    """
    Resolve parameter value - either direct value or @reference to component attribute.
    
    Parameters
    ----------
    value : any
        The parameter value from Excel (can be direct value or @reference)
    component_data : dict
        The component's data dictionary (node or edge attributes)
    param_name : str
        Name of the parameter being resolved
    component_id : str or tuple
        Identifier of the component (node id or edge tuple)
        
    Returns
    -------
    resolved_value : any
        The resolved parameter value
        
    Raises
    ------
    ValueError
        If referenced attribute does not exist in component_data
    """
    if isinstance(value, str) and value.startswith('@'):
        # It's a reference to a component attribute
        attr_name = value[1:]  # Remove '@' prefix
        if attr_name in component_data:
            return component_data[attr_name]
        else:
            raise ValueError(
                f"Component {component_id}: Parameter '{param_name}' references "
                f"non-existent attribute '@{attr_name}'"
            )
    else:
        # Direct value
        return value


def _load_template(template_path, logger):
    """
    Load and parse template file to extract parameter requirements.
    
    Parameters
    ----------
    template_path : str or Path
        Path to template file
    logger : logging.Logger
        Logger instance
        
    Returns
    -------
    main_parameters : list
        List of MAIN parameter names
    aux_parameters : list
        List of AUX parameter names
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    Exception
        If template parsing fails
    """
    logger.info(f"Loading template file: {template_path}")
    
    if not os.path.exists(template_path):
        error_msg = f"Template file not found: {template_path}"
        logger.error(error_msg)
        raise FileNotFoundError(error_msg)
    
    logger.info("Parsing template to extract parameter requirements")
    try:
        main_parameters, aux_parameters = parse_template_parameters(template_path, logger)
    except Exception as e:
        error_msg = f"Failed to parse template: {e}"
        logger.error(error_msg)
        raise
    
    logger.info(f"Template requires {len(main_parameters)} MAIN and {len(aux_parameters)} AUX parameters")
    logger.debug(f"MAIN: {main_parameters}")
    logger.debug(f"AUX: {aux_parameters}")
    
    return main_parameters, aux_parameters


def _load_excel(excel_path, excel_sheet_name, logger):
    """
    Load parameters from Excel file.
    
    Parameters
    ----------
    excel_path : str or Path or None
        Path to Excel file (optional)
    excel_sheet_name : str
        Name of the Excel sheet to load
    logger : logging.Logger
        Logger instance
        
    Returns
    -------
    excel_params : dict
        Dictionary of parameters from Excel (empty dict if excel_path is None)
    """
    excel_params = {}
    
    if excel_path is not None:
        try:
            logger.info(f"Loading parameters from Excel: {excel_path}")
            excel_params = load_component_parameters(excel_path, excel_sheet_name)
            logger.debug(f"Excel parameters loaded: {list(excel_params.keys())}")
        except Exception as e:
            warning_msg = f"Could not load Excel parameters: {e}"
            logger.warning(warning_msg)
            warnings.warn(warning_msg, UserWarning)
    else:
        logger.info("No Excel file provided, using only graph attributes")
    
    return excel_params


def _process_component_parameters(component_id, component_data, main_parameters, 
                                  aux_parameters, excel_params, logger):
    """
    Process MAIN and AUX parameters for a single component (node or edge).
    
    This is the core validation logic shared by all three assignment functions.
    
    Parameters
    ----------
    component_id : str or tuple
        Identifier of the component (node id or edge tuple)
    component_data : dict
        The component's data dictionary
    main_parameters : list
        List of required MAIN parameter names
    aux_parameters : list
        List of optional AUX parameter names
    excel_params : dict
        Dictionary of parameters from Excel
    logger : logging.Logger
        Logger for debug messages
        
    Returns
    -------
    missing_main : list
        List of missing MAIN parameters for this component
    missing_aux : set
        Set of missing AUX parameters for this component
    stats : dict
        Statistics dict with 'from_graph' and 'from_excel' counts
    """
    missing_main = []
    missing_aux = set()
    stats = {'from_graph': 0, 'from_excel': 0}
    
    # Process MAIN parameters
    for param in main_parameters:
        if param in component_data:
            # Parameter already in component - keep it
            stats['from_graph'] += 1
            logger.debug(f"  ✓ MAIN '{param}' found in component")
        elif param in excel_params:
            # Parameter not in component, but available in Excel - apply it
            try:
                resolved_value = resolve_parameter_value(
                    excel_params[param], component_data, param, component_id
                )
                component_data[param] = resolved_value
                stats['from_excel'] += 1
                source = f"@{excel_params[param][1:]}" if isinstance(excel_params[param], str) and excel_params[param].startswith('@') else "Excel"
                logger.debug(f"  ✓ MAIN '{param}' applied from {source}")
            except ValueError as e:
                # Reference resolution failed
                missing_main.append(param)
                logger.error(f"  ✗ MAIN '{param}': {e}")
        else:
            # Parameter missing - ERROR!
            missing_main.append(param)
            logger.error(f"  ✗ MAIN '{param}' not found")
    
    # Process AUX parameters
    for param in aux_parameters:
        if param in component_data:
            # Parameter already in component - keep it
            stats['from_graph'] += 1
            logger.debug(f"  ✓ AUX '{param}' found in component")
        elif param in excel_params:
            # Parameter not in component, but available in Excel - apply it
            try:
                resolved_value = resolve_parameter_value(
                    excel_params[param], component_data, param, component_id
                )
                component_data[param] = resolved_value
                stats['from_excel'] += 1
                source = f"@{excel_params[param][1:]}" if isinstance(excel_params[param], str) and excel_params[param].startswith('@') else "Excel"
                logger.debug(f"  ✓ AUX '{param}' applied from {source}")
            except ValueError as e:
                # Reference resolution failed - treat as missing AUX
                missing_aux.add(param)
                logger.warning(f"  ⚠ AUX '{param}' reference failed: {e}")
        else:
            # Parameter missing - will use Modelica default
            missing_aux.add(param)
            logger.debug(f"  ⚠ AUX '{param}' not provided - will use Modelica default")
    
    return missing_main, missing_aux, stats


def _aggregate_statistics(all_stats_list):
    """Aggregate statistics from multiple components."""
    total_stats = {'from_graph': 0, 'from_excel': 0}
    for stats in all_stats_list:
        total_stats['from_graph'] += stats['from_graph']
        total_stats['from_excel'] += stats['from_excel']
    return total_stats


def _check_and_report_results(component_type, component_count, all_missing_main, 
                              all_missing_aux, total_stats, logger):
    """
    Check if validation was successful and report results.
    
    Parameters
    ----------
    component_type : str
        Type of component ('edge', 'supply node', 'demand node')
    component_count : int
        Number of components processed
    all_missing_main : list of tuples
        List of (component_id, param) tuples for missing MAIN parameters
    all_missing_aux : set
        Set of AUX parameter names missing across all components
    total_stats : dict
        Aggregated statistics
    logger : logging.Logger
        Logger instance
        
    Raises
    ------
    ValueError
        If any MAIN parameters are missing
    """
    # Report summary
    logger.info(f"✓ Processed {component_count} {component_type}(s)")
    logger.info(f"  - Parameters from graph: {total_stats['from_graph']}")
    logger.info(f"  - Parameters from Excel: {total_stats['from_excel']}")
    
    # Summary of missing AUX parameters
    if all_missing_aux:
        missing_count = len(all_missing_aux)
        warning_msg = (
            f"{missing_count} AUX parameter(s) not provided for {component_type}s, "
            f"will use Modelica defaults: {', '.join(sorted(all_missing_aux))}"
        )
        logger.warning(warning_msg)
        warnings.warn(warning_msg, UserWarning)
    
    # Check if validation was successful
    if all_missing_main:
        error_count = len(all_missing_main)
        error_msg = (
            f"Validation FAILED: {error_count} missing MAIN parameter(s)\n"
            f"Missing parameters per {component_type}:\n"
        )
        for component_id, param in all_missing_main:
            error_msg += f"  - {component_type.capitalize()} {component_id}: '{param}'\n"
        error_msg += (
            f"\nFix suggestions:\n"
            f"  → If parameter varies per {component_type}: add to uesgraph attributes\n"
            f"  → If parameter is same for all {component_type}s: add to Excel sheet"
        )
        logger.error(error_msg)
        raise ValueError(error_msg)
    else:
        logger.info("✓ All MAIN parameters successfully validated and applied")


# ============================================================================
# MAIN ASSIGNMENT FUNCTIONS
# ============================================================================

def assign_pipe_parameters(uesgraph, template_path, excel_path=None, logger=None):
    """
    Assign parameters to pipe edges in the uesgraph according to the flow chart logic.
    
    This function follows the validation flow:
    1. Load and parse the template file
    2. Extract MAIN (required) and AUX (optional) parameters
    3. Load Excel parameters if provided
    4. For each edge individually:
       - Check MAIN parameters: in edge → keep, not in edge → try Excel, missing → ERROR
       - Check AUX parameters: in edge → keep, not in edge → try Excel, missing → WARNING
    5. Apply parameters from Excel where needed (never overwrite existing edge attributes)
    6. Support @ references: Excel values starting with @ are resolved to edge attributes
    
    Parameters
    ----------
    uesgraph : UESGraph
        The urban energy system graph object (modified in-place)
    template_path : str or Path
        Path to the pipe template file (.mako)
    excel_path : str or Path, optional
        Path to Excel file containing component parameters
        If None, only graph attributes are used
    logger : logging.Logger, optional
        Logger for status messages and warnings
        If None, creates a default logger
        
    Returns
    -------
    uesgraph : UESGraph
        The updated graph object (same as input, modified in-place)
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    ValueError
        If required MAIN parameters are missing for any edge
        
    Warns
    -----
    UserWarning
        If optional AUX parameters are missing (will use Modelica defaults)
    """
    if logger is None:
        logger = logging.getLogger(__name__)
    
    # Step 1: Load template file
    main_parameters, aux_parameters = _load_template(template_path, logger)
    
    # Step 2: Load Excel parameters if provided
    excel_params = _load_excel(excel_path, 'Pipes', logger)
    
    # Step 3: Process each edge individually
    total_edges = len(list(uesgraph.edges()))
    logger.info(f"Processing {total_edges} edge(s)...")
    
    all_missing_main = []
    all_missing_aux = set()
    all_stats = []
    
    for edge_idx, edge in enumerate(uesgraph.edges(), 1):
        edge_data = uesgraph.edges[edge]
        logger.debug(f"Processing edge {edge_idx}/{total_edges}: {edge}")
        
        missing_main, missing_aux, stats = _process_component_parameters(
            edge, edge_data, main_parameters, aux_parameters, 
            excel_params, logger
        )
        
        # Collect results
        all_missing_main.extend([(edge, param) for param in missing_main])
        all_missing_aux.update(missing_aux)
        all_stats.append(stats)
    
    # Step 4: Aggregate and report results
    total_stats = _aggregate_statistics(all_stats)
    _check_and_report_results(
        'edge', total_edges, all_missing_main, all_missing_aux, 
        total_stats, logger
    )
    
    return uesgraph


def assign_supply_parameters(uesgraph, template_path, excel_path=None, logger=None):
    """
    Assign parameters to supply nodes in the uesgraph according to the flow chart logic.
    
    This function follows the validation flow:
    1. Load and parse the template file
    2. Extract MAIN (required) and AUX (optional) parameters
    3. Load Excel parameters if provided
    4. For each supply node individually:
       - Check MAIN parameters: in node → keep, not in node → try Excel, missing → ERROR
       - Check AUX parameters: in node → keep, not in node → try Excel, missing → WARNING
    5. Apply parameters from Excel where needed (never overwrite existing node attributes)
    6. Support @ references: Excel values starting with @ are resolved to node attributes
    
    Parameters
    ----------
    uesgraph : UESGraph
        The urban energy system graph object (modified in-place)
    template_path : str or Path
        Path to the supply template file (.mako)
    excel_path : str or Path, optional
        Path to Excel file containing component parameters
        If None, only graph attributes are used
    logger : logging.Logger, optional
        Logger for status messages and warnings
        If None, creates a default logger
        
    Returns
    -------
    uesgraph : UESGraph
        The updated graph object (same as input, modified in-place)
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    ValueError
        If required MAIN parameters are missing for any supply node
        
    Warns
    -----
    UserWarning
        If optional AUX parameters are missing (will use Modelica defaults)
    """
    if logger is None:
        logger = logging.getLogger(__name__)
    
    
    # Step 1: Load Excel parameters
    excel_params = _load_excel(excel_path, 'Supply', logger)
    
    # Step 2: Load template file
    main_parameters, aux_parameters = _load_template(excel_params['template_path'], logger)

    # Step 3: Find supply nodes
    network_type = uesgraph.graph.get("network_type", "heating")
    is_supply_key = f"is_supply_{network_type}"
    
    supply_nodes = [
        node for node in uesgraph.nodelist_building
        if uesgraph.nodes[node].get(is_supply_key, False)
    ]
    
    if not supply_nodes:
        warning_msg = f"No supply nodes found (looking for '{is_supply_key}' = True)"
        logger.warning(warning_msg)
        warnings.warn(warning_msg, UserWarning)
        return uesgraph
    
    # Step 4: Process each supply node individually
    total_nodes = len(supply_nodes)
    logger.info(f"Processing {total_nodes} supply node(s)...")
    
    all_missing_main = []
    all_missing_aux = set()
    all_stats = []
    
    for node_idx, node in enumerate(supply_nodes, 1):
        node_data = uesgraph.nodes[node]
        logger.debug(f"Processing supply node {node_idx}/{total_nodes}: {node}")
        
        missing_main, missing_aux, stats = _process_component_parameters(
            node, node_data, main_parameters, aux_parameters,
            excel_params, logger
        )
        
        # Collect results
        all_missing_main.extend([(node, param) for param in missing_main])
        all_missing_aux.update(missing_aux)
        all_stats.append(stats)
    
    # Step 5: Aggregate and report results
    total_stats = _aggregate_statistics(all_stats)
    _check_and_report_results(
        'supply node', total_nodes, all_missing_main, all_missing_aux,
        total_stats, logger
    )
    
    return uesgraph


def assign_demand_parameters(uesgraph, template_path, excel_path=None, logger=None):
    """
    Assign parameters to demand nodes in the uesgraph according to the flow chart logic.
    
    This function follows the validation flow:
    1. Load and parse the template file
    2. Extract MAIN (required) and AUX (optional) parameters
    3. Load Excel parameters if provided
    4. For each demand node individually:
       - Check MAIN parameters: in node → keep, not in node → try Excel, missing → ERROR
       - Check AUX parameters: in node → keep, not in node → try Excel, missing → WARNING
    5. Apply parameters from Excel where needed (never overwrite existing node attributes)
    6. Support @ references: Excel values starting with @ are resolved to node attributes
    
    Parameters
    ----------
    uesgraph : UESGraph
        The urban energy system graph object (modified in-place)
    template_path : str or Path
        Path to the demand template file (.mako)
    excel_path : str or Path, optional
        Path to Excel file containing component parameters
        If None, only graph attributes are used
    logger : logging.Logger, optional
        Logger for status messages and warnings
        If None, creates a default logger
        
    Returns
    -------
    uesgraph : UESGraph
        The updated graph object (same as input, modified in-place)
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    ValueError
        If required MAIN parameters are missing for any demand node
        
    Warns
    -----
    UserWarning
        If optional AUX parameters are missing (will use Modelica defaults)
    """
    if logger is None:
        logger = logging.getLogger(__name__)
    
    # Step 1: Load template file
    main_parameters, aux_parameters = _load_template(template_path, logger)
    
    # Step 2: Load Excel parameters if provided
    excel_params = _load_excel(excel_path, 'Demands', logger)
    
    # Step 3: Find demand nodes (buildings that are NOT supply)
    network_type = uesgraph.graph.get("network_type", "heating")
    is_supply_key = f"is_supply_{network_type}"
    
    demand_nodes = [
        node for node in uesgraph.nodelist_building
        if not uesgraph.nodes[node].get(is_supply_key, False)
    ]
    
    if not demand_nodes:
        warning_msg = f"No demand nodes found (looking for buildings with '{is_supply_key}' = False)"
        logger.warning(warning_msg)
        warnings.warn(warning_msg, UserWarning)
        return uesgraph
    
    # Step 4: Process each demand node individually
    total_nodes = len(demand_nodes)
    logger.info(f"Processing {total_nodes} demand node(s)...")
    
    all_missing_main = []
    all_missing_aux = set()
    all_stats = []
    
    for node_idx, node in enumerate(demand_nodes, 1):
        node_data = uesgraph.nodes[node]
        logger.debug(f"Processing demand node {node_idx}/{total_nodes}: {node}")
        
        missing_main, missing_aux, stats = _process_component_parameters(
            node, node_data, main_parameters, aux_parameters,
            excel_params, logger
        )
        
        # Collect results
        all_missing_main.extend([(node, param) for param in missing_main])
        all_missing_aux.update(missing_aux)
        all_stats.append(stats)
    
    # Step 5: Aggregate and report results
    total_stats = _aggregate_statistics(all_stats)
    _check_and_report_results(
        'demand node', total_nodes, all_missing_main, all_missing_aux,
        total_stats, logger
    )
    
    return uesgraph

In [3]:
# =============================================================================
# NEUE EXCEL-BASIERTE PIPELINE TESTEN
# =============================================================================

import os
import pandas as pd
import logging
from uesgraphs.systemmodels.model_generation_pipeline import uesgraph_to_modelica

def create_dummy_demand_files(workspace):
    """Erstelle minimale Demand-Dateien für E15 Test"""
    print("Creating dummy demand files...")

    # Einfache 8760h Zeitreihe (1 Jahr, stündlich)
    hours = range(8760)

    # Heating demands für deine E15 Gebäude
    # Namen entsprechend deiner GeoJSON-Gebäude anpassen
    heating_data = pd.DataFrame({
        'hour': hours,
        'w11_1': [5000] * 8760,    # 5kW konstant
        'w11_2': [7000] * 8760,    # 7kW konstant  
        'w11_3': [4000] * 8760,    # 4kW konstant
        'w11_4': [6000] * 8760,    # 6kW konstant
        'supply1': [0] * 8760,     # Supply hat keine Demand
    })

    # DHW und Cooling (minimal für Test)
    dhw_data = heating_data.copy() * 0.3  # 30% von Heating
    cooling_data = heating_data.copy() * 0.0  # Kein Cooling für Test

    # Speichern
    heating_path = os.path.join(workspace, 'e15_heating_demands.csv')
    dhw_path = os.path.join(workspace, 'e15_dhw_demands.csv')
    cooling_path = os.path.join(workspace, 'e15_cooling_demands.csv')

    heating_data.to_csv(heating_path, index=False)
    dhw_data.to_csv(dhw_path, index=False)
    cooling_data.to_csv(cooling_path, index=False)

    print(f"  ✓ Heating demands: {heating_path}")
    print(f"  ✓ DHW demands: {dhw_path}")
    print(f"  ✓ Cooling demands: {cooling_path}")

    return {
        'heating': heating_path,
        'dhw': dhw_path,
        'cooling': cooling_path
    }

def create_dummy_ground_temp(workspace):
    """Erstelle Ground Temperature File für Test"""
    print("Creating dummy ground temperature file...")

    # Vereinfachte Ground Temperature (konstant 10°C)
    ground_data = pd.DataFrame({
        'depth_1m': [10.0] * 8760,     # 1m Tiefe
        'depth_2m': [12.0] * 8760,     # 2m Tiefe
        'depth_3m': [14.0] * 8760      # 3m Tiefe
    })

    ground_path = os.path.join(workspace, 'e15_ground_temp.csv')
    ground_data.to_csv(ground_path, index=False)

    print(f"  ✓ Ground temperature: {ground_path}")
    return ground_path


# =============================================================================
# HAUPTTEST: NEUE PIPELINE AUSFÜHREN
# =============================================================================

print("🚀 Starting Excel-based pipeline test for E15...")
print(f"Using existing graph with {graph.number_of_edges()} edges")

try:
    # 1. Test-Dateien erstellen
    print("\n📁 Step 1: Creating test files...")
    demand_paths = create_dummy_demand_files(workspace)
    ground_temp_path = create_dummy_ground_temp(workspace)
    excel_config_path = r"/home/leon/git/uesgraphs/uesgraphs/data/uesgraphs_parameters_template.xlsx"

    # 2. Neue Pipeline aufrufen
    print("\n🔧 Step 2: Running new Excel-based pipeline...")
    print("This will:")
    print("  - Load simulation settings from Excel 'Simulation' sheet")
    print("  - Assign demand data to graph nodes")
    print("  - Assign pipe parameters from Excel 'Pipes' sheet")
    print("  - Assign supply parameters from Excel 'Supply' sheet")
    print("  - Assign demand parameters from Excel 'Demands' sheet")
    print("  - Generate Modelica files")

    uesgraph_to_modelica(
        uesgraph=graph,                    # Dein bestehender E15 Graph
        simplification_level=0,           # Keine Vereinfachung für Test
        workspace=workspace,               # E15 workspace 
        sim_setup_path=excel_config_path, # Unsere Excel-Konfiguration
        input_heating=demand_paths['heating'],
        input_dhw=demand_paths['dhw'],
        input_cooling=demand_paths['cooling'],
        ground_temp_path=ground_temp_path,
        log_level=logging.INFO             # Mehr Details im Log
    )

    print("\n✅ SUCCESS: Excel-based pipeline completed!")
    print(f"📂 Check the 'models' folder in: {workspace}")
    print("🔍 Look for generated .mo files and simulation setup")

except Exception as e:
    print(f"\n❌ ERROR: Pipeline failed with: {e}")
    print("💡 Check the log files for detailed error information")
    raise

print("\n🎉 Excel-based pipeline test completed!")
print("📊 Compare results with previous pipeline outputs")



🚀 Starting Excel-based pipeline test for E15...
Using existing graph with 12 edges

📁 Step 1: Creating test files...
Creating dummy demand files...
  ✓ Heating demands: /home/leon/git/uesgraphs/workspace/e15/e15_heating_demands.csv
  ✓ DHW demands: /home/leon/git/uesgraphs/workspace/e15/e15_dhw_demands.csv
  ✓ Cooling demands: /home/leon/git/uesgraphs/workspace/e15/e15_cooling_demands.csv
Creating dummy ground temperature file...
  ✓ Ground temperature: /home/leon/git/uesgraphs/workspace/e15/e15_ground_temp.csv

🔧 Step 2: Running new Excel-based pipeline...
This will:
  - Load simulation settings from Excel 'Simulation' sheet
  - Assign demand data to graph nodes
  - Assign pipe parameters from Excel 'Pipes' sheet
  - Assign supply parameters from Excel 'Supply' sheet
  - Assign demand parameters from Excel 'Demands' sheet
  - Generate Modelica files
Logfile findable here: /tmp/ModelicaCodeGen_20251031_180839.log


/home/leon/git/uesgraphs/uesgraphs/systemmodels/model_generation_pipeline.py:1310: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(input_paths_dict[input_type],
/home/leon/git/uesgraphs/uesgraphs/systemmodels/model_generation_pipeline.py:1310: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(input_paths_dict[input_type],



❌ ERROR: Pipeline failed with: Failed to parse template: expected str, bytes or os.PathLike object, not float
💡 Check the log files for detailed error information


Exception: Failed to parse template: expected str, bytes or os.PathLike object, not float